In [11]:
import sys
import tweepy as tw
import pandas as pd

#sys.path.insert(0, '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco')
sys.path.insert(0, '/Users/will/Google Drive/workspace/Projetos/projetoCangaco')
#sys.path.insert(0, '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco/maria_bonita/dataset')
from credentials import API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET
#from minha_stream_listener import MinhaStreamListener


auth = tw.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tw.API(auth)

In [150]:
class StreamListenerPersonalizado(tw.StreamListener):
    def __init__(self, lista_resultados, limite=-1):
        tw.StreamListener.__init__(self)
        self.lista_resultados = lista_resultados
        self.limite = limite
        
    def __selecionar_chave_valor(self, lista_origem: list, chaves: list):
        lista_retorno = []
        for dicionario in lista_origem:
            dic_retorno = {}
            for chave, valor in dicionario.items():
                if chave in chaves: dic_retorno[chave] = valor
            lista_retorno.append(dic_retorno)
        return lista_retorno
                
    
    def on_status(self, status):
        
        eh_retweet = status.text.startswith('RT')
        
        localizacao_erro = 0
        
        try:            
            if not eh_retweet:
                
                qtd_capturados = len(self.lista_resultados) + 1
                atingiu_limite = qtd_capturados == self.limite
                if not atingiu_limite:
                    
                    print(qtd_capturados, (72 - len(str(qtd_capturados))) * '-') 
                    print(f'Usuário: {status.user.screen_name}')
                    print(f'Texto: {status.text}')
                    
                    dicionario_tweet = {}
                    
                    # CRIADO EM
                    dicionario_tweet["criado_em"] = status.created_at
                    
                    # RETWEET
                    dicionario_tweet["retweet"] = eh_retweet
                    
                    # TWEET ID
                    dicionario_tweet["tweet_id"] = status.id_str
                    
# CRIAR MÉTODO      # TWEET URL
                    dicionario_tweet['tweet_url'] = ''
                    
                    # TWEET TEXTO
                    tem_extended_tweet = hasattr(status, "extended_tweet")
                    dicionario_tweet["tweet_texto"] = status.extended_tweet['full_text'] if tem_extended_tweet else status.text
                    
                    # TWEET HASHTAGS
                    dicionario_tweet["tweet_hashtags"] = []
                    if tem_extended_tweet:
                        if len(status.extended_tweet['entities']['hashtags']):
                            dicionario_tweet["tweet_hashtags"] = [hashtag['text'] for hashtag in status.extended_tweet['entities']['hashtags']]
                            
                    else:
                        if len(status.entities['hashtags']):
                            dicionario_tweet["tweet_hashtags"] = [hashtag['text'] for hashtag in status.entities['hashtags']]
                    
                    # TWEET URLS EXTERNOS
                    dicionario_tweet["tweet_urls_externos"] = []
                    if tem_extended_tweet:
                        if len(status.extended_tweet['entities']['urls']):
                            dicionario_tweet["tweet_urls_externos"] = [url['expanded_url'] for url in status.extended_tweet['entities']['urls']]
                    else:
                        if len(status.entities['urls']):
                            dicionario_tweet["tweet_urls_externos"] = [url['expanded_url'] for url in status.entities['urls']]
                    
                    # TWEET USUÁRIOS MENCIONADOS
                    dicionario_tweet["tweet_usuarios_mencionados"] = []
                    if tem_extended_tweet:
                        if len(status.extended_tweet['entities']['user_mentions']):                        
                            dicionario_tweet["tweet_usuarios_mencionados"] = self.__selecionar_chave_valor(status.extended_tweet['entities']['user_mentions'], ['screen_name', 'id_str'])
                    else:
                        if len(status.entities['user_mentions']):
                            dicionario_tweet["tweet_usuarios_mencionados"] = self.__selecionar_chave_valor(status.entities['user_mentions'], ['screen_name', 'id_str'])
                            
                    # TWEET PLATAFORMA DE ORIGEM
                    dicionario_tweet["tweet_plataforma_origem"] = status.source[12:]
                    
                    # RESPOSTA AO TWEET
                    dicionario_tweet["resposta_ao_tweet"] = status.in_reply_to_status_id_str
                    
                    # RESPOSTA AO USUARIO
                    dicionario_tweet["resposta_ao_usuario"] = status.in_reply_to_screen_name
                    
                    # RESPOSTA AO USUARIO ID
                    dicionario_tweet["resposta_ao_usuario_id"] = status.in_reply_to_user_id_str
                    
                    # TWEET LÍNGUA
                    dicionario_tweet["tweet_lingua"] = status.lang# if hasattr(status, "language") else ""
                    
                    # TWEET JSON
                    dicionario_tweet["tweet_json"] = status._json
                    
                    # AUTOR NOME
                    dicionario_tweet["autor_nome"] = status.author.name
                    
                    # AUTOR SCREEN NAME
                    dicionario_tweet["autor_screen_name"] = status.author.screen_name
                    
                    # AUTOR ID
                    dicionario_tweet["autor_id"] = status.author.id_str
                    
# CRIAR MÉTODO      # AUTOR URL
                    dicionario_tweet["autor_url"] = ""
                    
                    # AUTOR VERIFICADO
                    dicionario_tweet["autor_verificado"] = status.author.verified
                
                    # AUTOR LOCALIZAÇÃO
                    dicionario_tweet["autor_localizacao"] = status.author.location
                    
                    # AUTOR DESCRICAO
                    dicionario_tweet["autor_descricao"] = status.author.description                    
                    
                    # AUTOR QUANTIDADE SEGUIDORES
                    dicionario_tweet["autor_qtd_seguidores"] = status.author.followers_count
                    
                    # AUTOR QUANTIDADE AMIGOS
                    dicionario_tweet["autor_qtd_amigos"] = status.author.friends_count
                    
                    # AUTOR QUANTIDADE TWEETS
                    dicionario_tweet["autor_qtd_tweets"] = status.author.statuses_count
                    
                    # AUTOR URL IMAGEM PERFIL
                    dicionario_tweet["autor_url_imagem_perfil"] = status.author.profile_image_url_https if hasattr(status, "profile_image_url") else ""
                    
                    # AUTOR URL IMAGEM CAPA
                    dicionario_tweet["autor_url_imagem_capa"] = status.author.profile_banner_url if hasattr(status, "profile_banner_url") else ""
                    
                    # AUTOR CRIADO EM
                    dicionario_tweet["autor_criado_em"] = status.author.created_at
                    
                    # AUTOR JSON
                    dicionario_tweet["autor_json"] = status.author._json
                    
                    # USUÁRIO NOME
                    dicionario_tweet["usuario_nome"] = status.user.name
                    
                    # USUÁRIO SCREEN NAME
                    dicionario_tweet["usuario_screen_name"] = status.user.screen_name
                    
                    # USUÁRIO ID
                    dicionario_tweet["usuario_id"] = status.user.id_str
                    
# CRIAR MÉTODO      # USUÁRIO URL
                    dicionario_tweet["usuario_url"] = ""
                    
                    # USUÁRIO VERIFICADO
                    dicionario_tweet["usuario_verificado"] = status.user.verified
                
                    # USUÁRIO LOCALIZAÇÃO
                    dicionario_tweet["usuario_localizacao"] = status.user.location
                    
                    # USUÁRIO DESCRICAO
                    dicionario_tweet["usuario_descricao"] = status.user.description                    
                    
                    # USUÁRIO QUANTIDADE SEGUIDORES
                    dicionario_tweet["usuario_qtd_seguidores"] = status.user.followers_count
                    
                    # USUÁRIO QUANTIDADE AMIGOS
                    dicionario_tweet["usuario_qtd_amigos"] = status.user.friends_count
                    
                    # USUÁRIO QUANTIDADE TWEETS
                    dicionario_tweet["usuario_qtd_tweets"] = status.user.statuses_count
                    
                    # USUÁRIO URL IMAGEM PERFIL
                    dicionario_tweet["usuario_url_imagem_perfil"] = status.user.profile_image_url_https if hasattr(status, "profile_image_url") else ""
                    
                    # USUÁRIO URL IMAGEM CAPA
                    dicionario_tweet["usuario_url_imagem_capa"] = status.user.profile_banner_url if hasattr(status, "profile_banner_url") else ""
                    
                    # USUÁRIO CRIADO EM
                    dicionario_tweet["usuario_criado_em"] = status.user.created_at
                    
                    # USUÁRIO JSON
                    dicionario_tweet["usuario_json"] = status.user._json

                    self.lista_resultados.append(dicionario_tweet)
    
                    print(72 * '-', end='\n\n')
        
                else:
                    return
        
        except BaseException as erro:
            print('on_status falhou:', str(erro), '- Posição erro:', localizacao_erro)
            if localizacao_erro == 1:
                print(status.extended_tweet['entities']['hashtags'])
            elif localizacao_erro == 2:
                print(status.entities['hashtags'])
            elif localizacao_erro == 3:
                print(status.extended_tweet['entities']['urls'])
            elif localizacao_erro == 4:
                print(status.entities['urls'])

In [151]:
#lista_ofensas = ['pedófila', 'pedófilo', 'pedofila', 'pedofilo', 'pedofilia']

lista_ofensas = ['bolsonaro']

In [155]:
lista_tweets = []

minha_stream = None
minha_stream = tw.Stream(auth=auth,
                         listener=MinhaStreamListener(lista_tweets, 50))

In [156]:
minha_stream.filter(languages=['pt'], track=lista_ofensas)

1 -----------------------------------------------------------------------
Usuário: DeiaTrevisan
Texto: @leandroruschel Bolsonaro não é Deus .... Ele não pode resolver todos os problemas do país em um ou dois anos.... E… https://t.co/nBtUDuRi72
------------------------------------------------------------------------

2 -----------------------------------------------------------------------
Usuário: Opif_Pires
Texto: @zsg_39 O que Bolsonaro tem com isso? Agora nós sabemos porque o Alexandre de Morais pegou ranço dele...
------------------------------------------------------------------------

3 -----------------------------------------------------------------------
Usuário: zze_carlos
Texto: @THAISVIANELLO Direita unida com Bolsonaro!
#FechadoComBolsonaro
------------------------------------------------------------------------

4 -----------------------------------------------------------------------
Usuário: MaisLigue
Texto: @BOLS4KUR4 Essas fotos, esses encontros...A direita do livro é

29 ----------------------------------------------------------------------
Usuário: MariaHelena_56
Texto: @PastorMalafaia Bolsonaro é Noé está construindo um barco, para salvar àqueles q nele embarcarem,  e confiar que se… https://t.co/UMtxJZIY7v
------------------------------------------------------------------------

30 ----------------------------------------------------------------------
Usuário: aboutgui
Texto: O marketeiro empenhado na mesma campanha que elegeu bolsonaro, inventam mentiras, fake news... é revoltante, chocante
------------------------------------------------------------------------

31 ----------------------------------------------------------------------
Usuário: FabioIng
Texto: @Tais009237611 @jairbolsonaro - COMUNISTAS e SOCIALISTAS são MENTIROSOS, CALUNIADORES e ASSASSINOS!

EM INGLÊS , BO… https://t.co/7wbqVJd0cm
------------------------------------------------------------------------

32 ----------------------------------------------------------------------
U

KeyboardInterrupt: 

In [159]:
dataset = pd.DataFrame(lista_tweets)
dataset

,criado_em,retweet,tweet_id,tweet_url,tweet_texto,tweet_hashtags,tweet_urls_externos,tweet_usuarios_mencionados,tweet_plataforma_origem,resposta_ao_tweet,...,usuario_verificado,usuario_localizacao,usuario_descricao,usuario_qtd_seguidores,usuario_qtd_amigos,usuario_qtd_tweets,usuario_url_imagem_perfil,usuario_url_imagem_capa,usuario_criado_em,usuario_json
0,2020-10-04 14:21:19,False,1312759710689513474,,@leandroruschel Bolsonaro não é Deus .... Ele ...,[],[],"[{'screen_name': 'leandroruschel', 'id_str': '...",iPhone,1312758314317611008,...,False,None,"Conservadora, Cristã, Patriota, Anti aborto, a...",204,450,9043,,,2009-09-14 22:10:07,"{'id': 74280110, 'id_str': '74280110', 'name':..."
1,2020-10-04 14:21:20,False,1312759714002989056,,@zsg_39 O que Bolsonaro tem com isso? Agora nó...,[],[],"[{'screen_name': 'zsg_39', 'id_str': '10246435...",Android,1312703344826253313,...,False,None,None,4567,5024,51054,,,2014-03-18 00:16:19,"{'id': 2395323132, 'id_str': '2395323132', 'na..."
2,2020-10-04 14:21:22,False,1312759720978124815,,@THAISVIANELLO Direita unida com Bolsonaro!\n#...,[FechadoComBolsonaro],[],"[{'screen_name': 'THAISVIANELLO', 'id_str': '5...",Android,1312758153692491776,...,False,None,None,665,1221,2311,,,2013-06-13 20:58:26,"{'id': 1514470105, 'id_str': '1514470105', 'na..."
3,2020-10-04 14:21:22,False,1312759722307616769,,"@BOLS4KUR4 Essas fotos, esses encontros...A di...",[],[],"[{'screen_name': 'BOLS4KUR4', 'id_str': '12875...",Android,1312743815782961152,...,False,None,None,1,0,292,,,2020-09-24 23:19:41,"{'id': 1309271041043320839, 'id_str': '1309271..."
4,2020-10-04 14:21:23,False,1312759725256323080,,@jairbolsonaro O cara tá ressuscitando assunto...,[],[],"[{'screen_name': 'jairbolsonaro', 'id_str': '1...",Android,1312758989046910977,...,False,None,Produtor de conteúdo original Nortista. Direto...,66,537,5453,,,2012-07-31 18:09:49,"{'id': 728919180, 'id_str': '728919180', 'name..."
5,2020-10-04 14:21:23,False,1312759726854283264,,@jairbolsonaro como seu eleitor exijo saber a ...,[],[],"[{'screen_name': 'jairbolsonaro', 'id_str': '1...",Android,None,...,False,"South Park, USA","Porra, Cartman!",217,548,14566,,,2019-12-16 16:32:35,"{'id': 1206612736920244224, 'id_str': '1206612..."
6,2020-10-04 14:21:25,False,1312759734848585731,,"Essa imagem é Asquerosa, Bolsonaro é um Vendid...",[BolsonaroTraidor],[],[],App,None,...,False,None,"Apaixonado Por Samba, Futebol E Conservador Di...",30,63,4530,,,2020-07-21 21:15:16,"{'id': 1285684753778331662, 'id_str': '1285684..."
7,2020-10-04 14:21:26,False,1312759739219148802,,@LuizSilvaNovo @MarcosASTZ @VemPraRua_br Até o...,[],[],"[{'screen_name': 'LuizSilvaNovo', 'id_str': '9...",Android,1312752678892638208,...,False,None,"Anti-Bolsopetismo,me chamar de isentão é um el...",34,41,2656,,,2020-06-12 13:36:29,"{'id': 1271436133013114882, 'id_str': '1271436..."
8,2020-10-04 14:21:27,False,1312759744227168256,,@AssumpcaD @jairbolsonaro O Presidente Bolsona...,[BolsonaroOrgulhaOBrasil],[],"[{'screen_name': 'AssumpcaD', 'id_str': '10577...",Android,1312359192334872577,...,False,"São Paulo, Brasil","Brasil acima de tudo, Deus acima de todos! 🇧🇷🇧...",3177,4989,9782,,,2018-12-20 18:17:59,"{'id': 1075817603237601281, 'id_str': '1075817..."
9,2020-10-04 14:21:27,False,1312759744097067008,,@_SC_Duda @SandraJbDIREITA Acho que esta é a i...,[],[],"[{'screen_name': '_SC_Duda', 'id_str': '110998...",Android,1312518895840849920,...,False,"Mato Grosso do Sul, Brasil",bunitao,43,106,1221,,,2020-05-28 20:17:31,"{'id': 1266101252146049024, 'id_str': '1266101..."


In [160]:
dataset.to_csv('./output/pedofilia.csv', sep=';', index=False, encoding='utf-8-sig')

In [47]:
dataset.texto.loc[[0]]

0    RT @CarlaZambelli38: Atenção, jovens, crianças...
Name: texto, dtype: object